#  EM-algorithm

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

## Data Preprocessing

### SciKitLearn digits

In [2]:
from sklearn import datasets
digits = datasets.load_digits()
x_digits = digits.data
y_digits = digits.target

Spliting the data set into 70% training data and 30% test data.

In [3]:
from sklearn.model_selection import train_test_split
x_digits_train, x_digits_test, y_digits_train, y_digits_test = train_test_split(x_digits, y_digits, test_size=0.3, random_state=42)
x_digits_train, x_digits_test = x_digits_train/16.0, x_digits_test/16.0

### EM-clustering against ground truth

In [4]:
import heapq
class EM_Gauss():
    def __init__(self, epsilon = 1e-2, threshold = 1e-3):
        self.epsilon = epsilon
        self.threshold = threshold

    # density function of the normal distribution
    def pdf_vector(self, x, mean, var):
        return 1 / np.sqrt(2 * np.pi * var) * np.exp(-1 / (2 * var) * (x - mean) ** 2)
    
    # fitting the model using EM algorithm
    def fit(self, data, target, labels):
        data = np.array(data)
        self.labels = labels
        nb_samples, nb_features = np.shape(data)
        self.priors = np.ones(labels)/labels
            
        # Initiliazing means and standard deviations
        self.means = np.empty([labels, nb_features])
        self.std = np.empty([labels, nb_features])
        
        for k in range(labels):
            self.means[k] = np.mean(data[target == k], axis = 0)
            self.std[k] = np.var(data[target == k], axis = 0)
        self.std += self.epsilon
        
        # Weights
        proba = np.empty([labels, nb_samples])
        weights = np.empty([labels, nb_samples])
        prev_priors = np.zeros(labels)
        
        while True:
            
            # E step
            for k in range(labels):
                proba[k,:] = np.prod(self.pdf_vector(data, self.means[k], self.std[k]), axis=1)
                weights[k, :] = self.priors[k] * proba[k,:]
            weights = weights / (np.sum(weights, axis = 0))

            # M step
            rk = np.sum(weights, axis = 1)
            prev_priors = self.priors
            self.priors = rk / nb_samples
                   
            # Updation means and std
            for k in range(labels):
                self.means[k] = np.sum(weights[k, :].reshape(-1,1) * data, axis = 0) / rk[k]
                self.std[k] = np.diag( ( weights[k, :].reshape(-1, 1) * (data - self.means[k]) ).T @ (data - self.means[k]) / rk[k] )
            self.std += self.epsilon
            
            # Condition of stop (comparing prior probabilities with previous priors)
            if (np.linalg.norm(prev_priors - self.priors) < self.threshold):
                break
        

    # Predicting the labels for given features
    def predict(self, test):
        x = np.array(test)
        length = x.shape[0]
        y_hat = np.zeros((length, ))  
        for i in range(length):
            proba = dict()
            probas = []
            for label in range(self.labels):          
                product = np.longdouble(-self.priors[label])
                for j in range(x.shape[1]):
                      product *= self.pdf_vector(x[i][j], self.means[label][j], math.sqrt(self.std[label][j]))
                heapq.heappush(probas, product)
                proba[product] = label
            
            y_hat[i] = proba[probas[0]]
        return y_hat
    
    def __repr__(self):
        return "EM_Gauss(epsilon={})".format(self.epsilon)
    def __str__(self):
        return "EM_Gauss(epsilon={})".format(self.epsilon)

In [5]:
classifier = EM_Gauss()
classifier.fit(x_digits_train, y_digits_train, 10)

In [6]:
y_hat = classifier.predict(x_digits_train)

In [7]:
from sklearn.metrics import completeness_score, homogeneity_score, adjusted_mutual_info_score

print("Completeness: ", completeness_score(y_digits_train, y_hat))
print("homogeneity: ", homogeneity_score(y_digits_train, y_hat))
print("mutual_info: ", adjusted_mutual_info_score(y_digits_train, y_hat))

Completeness:  0.7750072213204816
homogeneity:  0.7662787437951897
mutual_info:  0.7672811522523626


In [8]:
# Evaluation:
print("Confusion matrix:\n", confusion_matrix(y_digits_train, y_hat), "\n")
print("classification report:\n", classification_report(y_digits_train, y_hat), "\n")

Confusion matrix:
 [[124   0   0   0   1   0   0   0   0   0]
 [  0  99  10   0   0   1   8   0   0  14]
 [  1   7 114   3   0   0   0   1   4   0]
 [  0   1   0 117   0   1   0   6   3   1]
 [  0   3   0   0 111   0   0   4   3   0]
 [  2   0   0  10   0 102   0   0   1   1]
 [  0   1   0   0   0   0 127   0   0   0]
 [  0   1   0   0   0   0   0 121   2   0]
 [  0  32   2   4   0   1   5   1  85   1]
 [  3   1   0  50   0   1   0   8   3  55]] 

classification report:
               precision    recall  f1-score   support

           0       0.95      0.99      0.97       125
           1       0.68      0.75      0.71       132
           2       0.90      0.88      0.89       130
           3       0.64      0.91      0.75       129
           4       0.99      0.92      0.95       121
           5       0.96      0.88      0.92       116
           6       0.91      0.99      0.95       128
           7       0.86      0.98      0.91       124
           8       0.84      0.65    

### K-means against ground truth

In [440]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=10, random_state=0)
# fitting the model
cluster = model.fit(x_digits_train)

In [442]:
y_hat = model.predict(x_digits_train)

In [443]:
print("Completeness: ", completeness_score(y_digits_train, y_hat))
print("homogeneity: ", homogeneity_score(y_digits_train, y_hat))
print("mutual_info: ", adjusted_mutual_info_score(y_digits_train, y_hat))

Completeness:  0.7360914843284315
homogeneity:  0.7285326270567938
mutual_info:  0.7246048855466674


/home/nabil/anaconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


### EM against k-Means

In [462]:
class EM_Kmeans():
    def __init__(self, epsilon = 1e-2):
        self.epsilon = epsilon
    
    def fit(self, data, target, labels):
        data = np.array(data)
        self.labels = labels
        nb_samples, nb_features = np.shape(data)            
        self.means = np.empty([labels, nb_features])
        self.z = np.empty(nb_samples)

        for k in range(labels):
            self.means[k] = np.mean(data[target == k], axis = 0)
        
        prev_means = np.zeros((labels, nb_features))
        while np.linalg.norm(prev_means - self.means) > 1e-6:
            
            prev_means = np.array(self.means)
            # E step
            for i in range(nb_samples):
                distance = []
                index = dict()
                for k in range(labels):
                    norm = np.linalg.norm(data[i] - self.means[k])
                    heapq.heappush(distance, norm**2)
                    index[norm**2] = k
                self.z[i] = index[distance[0]]
            # M step 
            for k in range(labels):
                self.means[k] = np.mean(data[self.z == k], axis = 0)

    def predict(self, test):
        x = np.array(test)
        length = x.shape[0]
        y_hat = np.zeros((length, ))  
        for i in range(length):
            proba = dict()
            probas = []
            for label in range(self.labels):          
                product = np.longdouble(-self.priors[label])
                for j in range(x.shape[1]):
                      product *= self.pdf_vector(x[i][j], self.means[label][j], math.sqrt(self.std[label][j]))
                heapq.heappush(probas, product)
                proba[product] = label
            
            y_hat[i] = proba[probas[0]]
        return y_hat
    
    def __repr__(self):
        return "EM_Kmeans(epsilon={})".format(self.epsilon)
    def __str__(self):
        return "EM_Kmeans(epsilon={})".format(self.epsilon)

In [472]:
classifier = EM_Kmeans()
classifier.fit(x_digits_train, y_digits_train, 10)

In [464]:
y_hat = model.predict(x_digits_train)

In [465]:
print("Completeness: ", completeness_score(y_digits_train, y_hat))
print("homogeneity: ", homogeneity_score(y_digits_train, y_hat))
print("mutual_info: ", adjusted_mutual_info_score(y_digits_train, y_hat))

Completeness:  0.7360914843284315
homogeneity:  0.7285326270567938
mutual_info:  0.7246048855466674


/home/nabil/anaconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [473]:
# classifier = EM_Kmeans()
# classifier.fit(x_digits_test, y_digits_test, 10)
y_hat = model.predict(x_digits_test)
print("Completeness: ", completeness_score(y_digits_test, y_hat))
print("homogeneity: ", homogeneity_score(y_digits_test, y_hat))
print("mutual_info: ", adjusted_mutual_info_score(y_digits_test, y_hat))

Completeness:  0.769142602293317
homogeneity:  0.7570724943188667
mutual_info:  0.7483708732266358


/home/nabil/anaconda3/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
